Work done so far: change all data types, drop some corrupted rows with at start_time and end_time
Suggestion for handling null values at start_station_id: we may want to make use of latitude and longtitude? notice below when 
i sort the dataframe by bike_id, it shows the route that the one bike commute. So maybe find out which row has the same latitude/longtitude which the missing value row and 
and insert the id accordingly.


In [1]:
# import all relevant libraries
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random

import data 

In [2]:
bikeRen = pd.read_csv("baywheels_2019", encoding = "ISO-8859-1") 
bikeRen.head(20)

C:\Users\riven\AppData\Local\Temp/ipykernel_6968/4202054314.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  bikeRen = pd.read_csv("baywheels_2019", encoding = "ISO-8859-1")


,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,0,2019-10-28 09:46:47,2019-10-28 09:51:05,50.0,37.780526,-122.390288,453.0,37.777934,-122.396973,12424.0
1,1,2019-08-10 19:31:02,2019-08-10 19:35:05,285.0,37.783521,-122.431158,74.0,37.776435,-122.426244,1718.0
2,2,2019-05-27 13:18:58,2019-05-27 13:46:37,10.0,37.795393,-122.40477,399.0,37.802636,-122.436289,608.0
3,3,2019-07-24 22:47:58,2019-07-24 22:52:48,106.0,37.763242,-122.430675,72.0,37.772406,-122.43565,551642.0
4,4,2019-04-28 17:26:22,2019-04-28 17:31:54,80.0,37.775235,-122.397437,50.0,37.780526,-122.390288,1103.0
5,5,2019-04-17 19:39:05,2019-04-17 19:49:26,100.0,37.7671,-122.410662,74.0,37.776435,-122.426244,1473.0
6,6,2019-02-26 08:11:01,2019-02-26 08:22:58,67.0,37.776639,-122.395526,8.0,37.799953,-122.398525,4984.0
7,7,2019-02-02 09:23:15,2019-02-02 09:34:20,44.0,37.781074,-122.411738,55.0,37.777053,-122.429558,2666.0
8,8,2019-05-07 12:08:01,2019-05-07 12:12:00,9.0,37.798572,-122.400869,6.0,37.80477,-122.403234,1961.0
9,9,2019-09-04 21:58:19,2019-09-04 22:09:07,86.0,37.769305,-122.426826,39.0,37.778999,-122.436861,9717.0


In [10]:
print(bikeRen.info())
print(bikeRen.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Unnamed: 0         int64 
 1   start_time         object
 2   end_time           object
 3   start_station_id   object
 4   start_station_lat  object
 5   start_station_lon  object
 6   end_station_id     object
 7   end_station_lat    object
 8   end_station_lon    object
 9   bike_id            object
dtypes: int64(1), object(9)
memory usage: 191.3+ MB
None
         Unnamed: 0
count  2.507003e+06
mean   1.253501e+06
std    7.237096e+05
min    0.000000e+00
25%    6.267505e+05
50%    1.253501e+06
75%    1.880252e+06
max    2.507002e+06


In [11]:
bikeRen.isna().sum()

Unnamed: 0               0
start_time               0
end_time                 0
start_station_id     80734
start_station_lat        0
start_station_lon        0
end_station_id       82902
end_station_lat          0
end_station_lon          0
bike_id                  0
dtype: int64

In [14]:
bikeRen["bike_id"]= pd.to_numeric(bikeRen["bike_id"], errors='coerce')

print(bikeRen.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   start_time         object 
 2   end_time           object 
 3   start_station_id   object 
 4   start_station_lat  object 
 5   start_station_lon  object 
 6   end_station_id     object 
 7   end_station_lat    object 
 8   end_station_lon    object 
 9   bike_id            float64
dtypes: float64(1), int64(1), object(8)
memory usage: 191.3+ MB
None


In [20]:
 bikeRen["bike_id"]= bikeRen["bike_id"].astype('Int64')
 print(bikeRen.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Unnamed: 0         int64 
 1   start_time         object
 2   end_time           object
 3   start_station_id   object
 4   start_station_lat  object
 5   start_station_lon  object
 6   end_station_id     object
 7   end_station_lat    object
 8   end_station_lon    object
 9   bike_id            Int64 
dtypes: Int64(1), int64(1), object(8)
memory usage: 193.7+ MB
None


In [23]:
bikeRen["start_station_lat"]= pd.to_numeric(bikeRen["start_station_lat"], errors='coerce')
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   start_time         object 
 2   end_time           object 
 3   start_station_id   object 
 4   start_station_lat  float64
 5   start_station_lon  object 
 6   end_station_id     object 
 7   end_station_lat    object 
 8   end_station_lon    object 
 9   bike_id            Int64  
dtypes: Int64(1), float64(1), int64(1), object(7)
memory usage: 193.7+ MB


In [26]:
bikeRen["start_station_lon"]=pd.to_numeric(bikeRen["start_station_lon"], errors='coerce')
bikeRen["end_station_lat"]=pd.to_numeric(bikeRen["end_station_lat"], errors='coerce')
bikeRen["end_station_lon"]=pd.to_numeric(bikeRen["end_station_lon"], errors='coerce')
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   start_time         object 
 2   end_time           object 
 3   start_station_id   object 
 4   start_station_lat  float64
 5   start_station_lon  float64
 6   end_station_id     object 
 7   end_station_lat    float64
 8   end_station_lon    float64
 9   bike_id            Int64  
dtypes: Int64(1), float64(4), int64(1), object(4)
memory usage: 193.7+ MB


In [30]:
bikeRen["start_station_id"] =pd.to_numeric(bikeRen["start_station_id"], errors='coerce')
bikeRen["start_station_id"]= bikeRen["start_station_id"].astype("Int64")
bikeRen["end_station_id"] =pd.to_numeric(bikeRen["end_station_id"], errors='coerce')
bikeRen["end_station_id"]= bikeRen["end_station_id"].astype("Int64")

In [31]:
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507003 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   start_time         object 
 2   end_time           object 
 3   start_station_id   Int64  
 4   start_station_lat  float64
 5   start_station_lon  float64
 6   end_station_id     Int64  
 7   end_station_lat    float64
 8   end_station_lon    float64
 9   bike_id            Int64  
dtypes: Int64(3), float64(4), int64(1), object(2)
memory usage: 198.4+ MB


In [71]:
bikeRen.isnull().sum()

Unnamed: 0               0
start_time               0
end_time                 0
start_station_id     80743
start_station_lat        9
start_station_lon        9
end_station_id       82911
end_station_lat          9
end_station_lon          9
bike_id                 10
dtype: int64

In [90]:
mask = bikeRen["start_time"].str.contains('Failure')

result = bikeRen[mask]
result



,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id


note that i dropped all the rows with "failure" values for start_time and end_time
but i dont know how to drop them all at one :)
so i dropped them one by one.You may have a better idea dropping them :) otherwise run the result line to see the corrupted rows, and drop them by id


In [88]:
bikeRen = bikeRen.drop(2249494)

In [91]:
bikeRen["start_time"]= pd.to_datetime(bikeRen["start_time"],format ='%Y-%m-%d %H:%M:%S')

In [92]:
bikeRen["end_time"]= pd.to_datetime(bikeRen["end_time"],format ='%Y-%m-%d %H:%M:%S')

In [93]:
bikeRen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2506991 entries, 0 to 2507002
Data columns (total 10 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Unnamed: 0         int64         
 1   start_time         datetime64[ns]
 2   end_time           datetime64[ns]
 3   start_station_id   Int64         
 4   start_station_lat  float64       
 5   start_station_lon  float64       
 6   end_station_id     Int64         
 7   end_station_lat    float64       
 8   end_station_lon    float64       
 9   bike_id            Int64         
dtypes: Int64(3), datetime64[ns](2), float64(4), int64(1)
memory usage: 217.6 MB


In [96]:
bikeRen.head(10)

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,0,2019-10-28 09:46:47,2019-10-28 09:51:05,50,37.780526,-122.390288,453,37.777934,-122.396973,12424
1,1,2019-08-10 19:31:02,2019-08-10 19:35:05,285,37.783521,-122.431158,74,37.776435,-122.426244,1718
2,2,2019-05-27 13:18:58,2019-05-27 13:46:37,10,37.795393,-122.404770,399,37.802636,-122.436289,608
3,3,2019-07-24 22:47:58,2019-07-24 22:52:48,106,37.763242,-122.430675,72,37.772406,-122.435650,551642
4,4,2019-04-28 17:26:22,2019-04-28 17:31:54,80,37.775235,-122.397437,50,37.780526,-122.390288,1103
5,5,2019-04-17 19:39:05,2019-04-17 19:49:26,100,37.767100,-122.410662,74,37.776435,-122.426244,1473
6,6,2019-02-26 08:11:01,2019-02-26 08:22:58,67,37.776639,-122.395526,8,37.799953,-122.398525,4984
7,7,2019-02-02 09:23:15,2019-02-02 09:34:20,44,37.781074,-122.411738,55,37.777053,-122.429558,2666
8,8,2019-05-07 12:08:01,2019-05-07 12:12:00,9,37.798572,-122.400869,6,37.804770,-122.403234,1961
9,9,2019-09-04 21:58:19,2019-09-04 22:09:07,86,37.769305,-122.426826,39,37.778999,-122.436861,9717


In [106]:
bikeRen = bikeRen.sort_values(by='bike_id')
bikeRen.head(10)
bikeRen[bikeRen["start_station_id"].isnull()]

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
2111297,2111297,2019-03-16 19:13:27,2019-03-16 19:19:06,<NA>,37.410000,-121.960000,<NA>,37.420000,-121.960000,3975
1957152,1957152,2019-03-14 14:35:27,2019-03-14 14:42:14,<NA>,37.420000,-121.950000,<NA>,37.410000,-121.960000,3975
2298364,2298364,2019-02-27 14:06:51,2019-02-27 14:13:31,<NA>,37.410000,-121.950000,<NA>,37.410000,-121.960000,3975
829344,829344,2019-02-27 13:37:01,2019-02-27 13:43:44,<NA>,37.410000,-121.950000,<NA>,37.410000,-121.950000,3975
634783,634783,2019-03-07 14:40:30,2019-03-07 14:44:49,<NA>,37.410000,-121.940000,<NA>,37.420000,-121.940000,3975
...,...,...,...,...,...,...,...,...,...,...
1071870,1071870,2019-12-03 14:21:13,2019-12-03 14:24:56,<NA>,37.775992,-122.392654,<NA>,37.775967,-122.392704,999941
767140,767140,2019-12-11 15:28:11,2019-12-11 15:29:59,<NA>,37.775930,-122.392638,<NA>,37.775869,-122.392646,999941
813813,813813,2019-12-08 01:15:35,2019-12-08 01:16:41,<NA>,37.775925,-122.392526,<NA>,37.776050,-122.392549,999941
1465980,1465980,2019-12-19 16:50:20,2019-12-19 16:52:07,<NA>,37.775904,-122.392551,<NA>,37.775879,-122.392585,999941
